In [ ]:
# Warning: Normally its '  but in the text is ’
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    # general
    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase

def sentence2Vec(dataset,model,wordDim,maxLength):
    sentenceVec = []
    for sentence in dataset:
        wordVec = []
        for vocab in sentence:
            if vocab == marked_token:
                wordVec.append(np.ones(wordDim))
            else:
                try:
                    vec =  model[vocab]
                    wordVec.append(vec)
                except :
                    wordVec.append(np.zeros(wordDim))
        sentenceVec.append(wordVec)

    sentenceVec = keras.preprocessing.sequence.pad_sequences(sentenceVec, maxlen=maxLength, dtype='float', padding='post', truncating='post', value=0.0)
    return sentenceVec 


# High frequency vocab dictionary 
def getHighFrequencyVocab(frequentVocabLimit,mergeList):
    fdist1 = FreqDist(mergeList)
    frequentVocab = fdist1.most_common(frequentVocabLimit)
    frequentDic = {}
    for vocab in frequentVocab:
        frequentDic[vocab[0]] = True
    return frequentDic


def generateMaskedSentence(processedSentence,frequentDic):
    for sentence in processedSentence:
        for i in range(len(sentence)):
            if sentence[i] not in frequentDic.keys():
                sentence[i] = marked_token
    return processedSentence

def generateIndexForDataset(dataSize):
    trainIndexList = []
    data_index = []

    for i in range(dataSize-1):
        x = randint(0,dataSize-1)
        y = randint(0,dataSize-1)
        z = i+1 
        while(x == y or x==i or x==z or y == i or y==z ):
            x = randint(0,dataSize-1)
            y = randint(0,dataSize-1)
        index = ([[x,0],[y,0],[z,1]])
        random.shuffle(index)
        trainIndexList.append([i,index])   

    for row in trainIndexList:
        train_x = row[0]
        y_index = row[1]
        y_output = []
        x_masked = []
        for index in y_index:
            y_output.append(index[1])
            x_masked.append(index[0])
        data_index.append([train_x,x_masked,y_output])

    random.shuffle(data_index)

    return data_index


def loadEmbeddingModel(embeddingPath,word2VecFile,gloveFile,convertGloveToWordVec):
    word2vec_output_file = embeddingPath + word2VecFile
    glove_input_file = embeddingPath + gloveFile

    if convertGloveToWordVec:
        glove2word2vec(glove_input_file, word2vec_output_file)

    # load the Stanford GloVe model
    model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
    return model



def loadFile(textPath,orgFile):
    with open(textPath +  orgFile ,encoding="utf-8") as fp:
        lines = fp.readlines()
    # Remove the empty line 
    lines = [e for e in lines if e not in {'\n'}]
    
    return lines
    
def preprocessing(lines): 
    newLine = []
    for line in lines:
        # Decapitalize: Conver to lower case first. 
        line = line.lower()
        # Delete url 
        line = re.sub(r"http\S+", "link", line)
        line = re.sub(r"\S+html", "link", line)
        line = re.sub(r"\S+.com$", "link", line)
        line = re.sub(r"\S+.jpg$", "photo", line)
        line = decontracted(line)

        '''
        ignore:  * { } \  < > 
        Splite based on : ! . ,  &  # ' $ 
        line = re.findall(r"[\w']+|[().,:!?;'$&]", line)
        '''

        line = re.findall(r"[\w']+|[().,:!?;'$&]", line)

        newLine.append(line)
    
    mergeList = []
    for i in newLine:
        mergeList += i   
    return newLine, mergeList

## Read file & Prepocessing

In [ ]:
import pickle  
from nltk import FreqDist
import re
import randomfa
from random import randint
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import numpy as np 

import keras
# import tensorflow as tf
# from keras.models import Model
# from keras.layers import Dense, Bidirectional,Input, Dropout, Flatten, concatenate, dot, GaussianDropout, Activation, GRU
# from keras.regularizers import l1, l2
import keras.backend as kb
from keras.callbacks import ModelCheckpoint

import Model

embeddingPath = "Embedding/"
convertGloveToWordVec = True  # The first time use Glove need to convert to Word2Vec format
gloveFile = 'glove.840B.300d.txt'  
word2VecFile =  'word2vec.6B.50d.txt'


wordDim = 300 

textPath = "text/"
orgFile = "all.txt"
processedFile = "process.txt"
maskedFile = "masked.txt"

frequentVocabLimit = 3500
marked_token = "MASKED_TOKEN"
trainingSize = 0
maxLength = 50
dataSize = 2000

needProcessData = True 

In [ ]:
if needProcessData:
    # Load the data, remove the empty line
    sentenceList = loadFile(textPath,orgFile)
    # Remove unwanted feature, proprocessing, return wordList
    processedLine, vocabList = preprocessing(sentenceList)

    dataSize = len(processedLine)
    # Get the high frequency vocabulary 
    frequenentVocabDic =  getHighFrequencyVocab(frequentVocabLimit,vocabList)

    # Save processed data
    with open(textPath +  processedFile, "wb") as internal_filename:
        pickle.dump(processedLine, internal_filename)

    # Generate the masked dataset
    maskedLine =  generateMaskedSentence(processedLine,frequenentVocabDic)

    # Save masked data
    with open(textPath +  maskedFile, "wb") as internal_filename:
        pickle.dump(maskedLine, internal_filename)
    
# Generate the index for the dataset 
dataIndex = generateIndexForDataset(dataSize)

# Read the data 
with open(textPath + processedFile, "rb") as input_file:
    processedLine = pickle.load(input_file)
with open(textPath + maskedFile , "rb") as input_file:
    maskedSentence = pickle.load(input_file)
    
embeddingMatrix = loadEmbeddingModel(embeddingPath,word2VecFile,gloveFile,convertGloveToWordVec)


In [ ]:
sentenceVec =  sentence2Vec(processedLine,embeddingMatrix,wordDim,maxLength)
maskedSentenceVec = sentence2Vec(processedLine,embeddingMatrix,wordDim,maxLength)

embeddingMatrix = 0 

In [ ]:
# Generate input_X, input_masked_X, input_Y
input_X = []
input_masked_first = []
input_masked_second = []
input_masked_third = []
input_Y = []
for index in dataIndex:
    input_X.append(sentenceVec[index[0]])
    input_Y.append(index[2])
    input_masked_first.append(maskedSentenceVec[index[1][0]])
    input_masked_second.append(maskedSentenceVec[index[1][1]])
    input_masked_third.append(maskedSentenceVec[index[1][2]])
    
input_X = np.array(input_X)
input_masked_first = np.array(input_masked_first)
input_masked_second = np.array(input_masked_second)
input_masked_third = np.array(input_masked_third)
input_Y = np.array(input_Y)

In [ ]:
length = int(0.99 * len(input_X))

input_X_train = input_X[0:length]
input_X_val = input_X[length:]
input_X = 0 

input_masked_first_train = input_masked_first[0:length]
input_masked_first_val = input_masked_first[length:]
input_masked_first = 0

input_masked_second_train = input_masked_second[0:length]
input_masked_second_val = input_masked_second[length:]
input_masked_second = 0

input_masked_third_train = input_masked_third[0:length]
input_masked_third_val = input_masked_third[length:]
input_masked_third = 0 

input_Y_train = input_Y[0:length]
input_Y_val = input_Y[length:]
input_Y = 0

## Modeling  

In [ ]:
num_neurons = [wordDim*2,wordDim*2]
kernel_reg = 0.001
batch_size = 512
epochs = 5000

merge_mode = 'concat'
extractLayer = ['X_input_second','X_input_mask_2']
checkpointPath = "checkpoints/weights-epoch-{epoch:02d}-loss-{loss:.4f}-val_loss-{val_loss:.4f}.hdf5"
checkpoint = "checkpoints/weights-epoch-02-loss-11.88-val_loss-11.56.hdf5"
restoreCheckpoint = False

In [ ]:
input_X = np.zeros((10,maxLength,wordDim))
model = Model.buildModel(input_X,kernel_reg,num_neurons,merge_mode)
model.compile( loss='mse', optimizer='adam')
if restoreCheckpoint:
    model.load_weights(checkpoint)
model.summary()

In [ ]:
checkpoint =  ModelCheckpoint(checkpointPath, monitor=('val_loss'), verbose=0, 
                              save_best_only=False, save_weights_only=False, mode='auto', period=50)

model.fit([input_X_train,input_masked_first_train,input_masked_second_train,input_masked_third_train],input_Y_train,
          validation_data=([input_X_val,input_masked_first_val,input_masked_second_val,input_masked_third_val],input_Y_val),
          epochs=epochs,batch_size=batch_size,callbacks=[checkpoint])

In [ ]:
output =  Model.extractHiddenState(extractLayer,model,np.zeros((10,maxLength,wordDim)))

In [ ]:
output =  np.concatenate((output[0],output[1]),axis=1)

## Extract hidden layer 

In [ ]:
output =  Model.extractHiddenState(extractLayer,model,input_X)
results1  =  np.concatenate((output[0],output[1]),axis=1)

In [ ]:
# Get the hidden layer for SentEval 
x =  np.zeros((10,maxLength,wordDim))
model = buildModel(x,kernel_reg,num_neurons,merge_mode)
model.load_weights(checkpoint)
output = extractHiddenState(layerName=extractLayer,model=model, predict_input=input_X)